In [2]:
#Indian Stock Market 10-Year Beginner-Friendly Sector Investment Advisor
# Focus: Top 5 Stocks Per Sector (Beginner Mode, No Graphs)
import warnings
import time
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore", category=FutureWarning)

START_DATE = (datetime.today() - timedelta(days=365 * 10)).strftime('%Y-%m-%d')
END_DATE = datetime.today().strftime('%Y-%m-%d')
RISK_FREE_RATE = 0.05

# SECTOR-WISE COMPANIES
SECTOR_COMPANIES = {
    "IT": ["TCS", "INFY", "HCLTECH", "TECHM", "WIPRO"],
    "BANKING": ["HDFCBANK", "ICICIBANK", "AXISBANK", "KOTAKBANK", "SBIN", "INDUSINDBK"],
    "FMCG": ["HINDUNILVR", "ITC", "NESTLEIND", "BRITANNIA", "DABUR"],
    "AUTO": ["MARUTI", "M&M", "BAJAJ-AUTO", "HEROMOTOCO", "EICHERMOT", "TATAMOTORS"],
    "ENERGY": ["RELIANCE", "ONGC", "NTPC", "POWERGRID", "BPCL"],
    "PHARMA": ["SUNPHARMA", "CIPLA", "DRREDDY", "DIVISLAB", "AUROPHARMA"],
    "METAL": ["TATASTEEL", "HINDALCO", "JSWSTEEL", "COALINDIA", "NMDC"],
    "FINANCIAL": ["BAJFINANCE", "BAJAJFINSV", "HDFCLIFE", "SBILIFE", "ICICIPRULI"],
    "INFRA": ["LT", "ULTRACEMCO", "GRASIM", "ADANIPORTS", "ADANIENT"],
    "CONSUMPTION": ["TITAN", "ASIANPAINT", "TATACONSUM", "VOLTAS", "PIDILITIND"]
}

print("💼 Sectoral & Thematic Indices (NSE India)\n")

print("🏦 Financial & Banking\nNIFTY Bank\nNIFTY Private Bank\nNIFTY PSU Bank\nNIFTY Financial Services\nNIFTY Financial Services 25/50\n")
print("💻 Technology & IT\nNIFTY IT\nNIFTY Digital Index (Thematic – focuses on digital transformation companies)\n")
print("⚙️ Infrastructure & Industrial\nNIFTY Infrastructure\nNIFTY PSE (Public Sector Enterprises)\nNIFTY CPSE (Central Public Sector Enterprises)\nNIFTY Infrastructure Debt Fund\n")
print("🛢️ Energy & Resources\nNIFTY Energy\nNIFTY Oil & Gas\nNIFTY Commodities\nNIFTY Metal\n")
print("🏠 Consumer & Lifestyle\nNIFTY FMCG\nNIFTY Consumer Durables\nNIFTY India Consumption\n")
print("🏗️ Manufacturing & Engineering\nNIFTY Auto\nNIFTY Industrials\nNIFTY Manufacturing\n")
print("💊 Healthcare\nNIFTY Healthcare Index\nNIFTY Pharma\n")
print("🌱 Thematic & Emerging Trends\nNIFTY India Digital Index\nNIFTY EV & New Age Automotive Index\nNIFTY100 ESG Sector Leaders\nNIFTY ESG 50\nNIFTY Commodities\nNIFTY MNC\nNIFTY Dividend Opportunities 50\nNIFTY High Beta 50\nNIFTY Low Volatility 50\n")

print("If you just want to invest in a specific industry, the most common beginner-friendly NSE sectoral indices are:\n")
print("🏦 NIFTY Bank\n💻 NIFTY IT\n🏠 NIFTY FMCG\n🚗 NIFTY Auto\n⚡ NIFTY Energy\n💊 NIFTY Pharma\n⛓️ NIFTY Metal\n💰 NIFTY Financial Services\n🏗️ NIFTY Infra\n🍴 NIFTY Consumption\n")
print("👉 If you want to invest in different sectors other than the suggested ones, enter the name from the above.\n")

def download_adj_close(tickers):
    adj = pd.DataFrame()
    print(f"\n📥 Downloading {len(tickers)} tickers... Please wait.")
    for t in tickers:
        t_symbol = t if t.endswith(".NS") else t + ".NS"
        success = False
        for attempt in range(3):
            try:
                data = yf.download(t_symbol, start=START_DATE, end=END_DATE, progress=False)
                if "Adj Close" in data.columns:
                    adj[t] = data["Adj Close"]
                elif "Close" in data.columns:
                    adj[t] = data["Close"]
                else:
                    num_cols = data.select_dtypes(include="number").columns
                    if len(num_cols) > 0:
                        adj[t] = data[num_cols[0]]
                success = True
                break
            except Exception as e:
                print(f"⚠️ Retry {attempt + 1}/3 for {t} due to: {e}")
                time.sleep(3)
        if not success:
            print(f"❌ Failed to download {t_symbol} after 3 attempts.")
    adj = adj.ffill().bfill()
    adj = adj.dropna(axis=1, thresh=int(0.8 * len(adj)))
    if adj.empty:
        print("⚠️ No data could be retrieved for this sector.\n")
    else:
        print(f"✅ Successfully downloaded {len(adj.columns)} tickers.\n")
    return adj

def compute_cagr(series):
    s = series.dropna()
    if s.empty: return np.nan
    years = (s.index[-1] - s.index[0]).days / 365.25
    return (s.iloc[-1] / s.iloc[0])**(1/years) - 1

def downside_std(returns):
    neg = returns[returns < 0]
    return neg.std() if not neg.empty else 0

def rolling_recovery_days(price):
    s = price.dropna()
    if s.empty: return np.nan
    peak, last_peak = s.iloc[0], s.index[0]
    max_days = 0
    for date, val in s.items():
        if val >= peak:
            peak = val
            last_peak = date
        else:
            days_under = (date - last_peak).days
            if days_under > max_days: max_days = days_under
    return max_days

# METRIC CALCULATION
def calculate_metrics(adj):
    returns = adj.pct_change().dropna()
    nifty_ref = yf.download("^NSEI", start=START_DATE, end=END_DATE, progress=False)
    nifty_series = nifty_ref.get("Adj Close", nifty_ref.select_dtypes("number").iloc[:, 0])
    nifty_ret = nifty_series.pct_change().dropna()
    results = {}
    for sym in adj.columns:
        px = adj[sym]
        ret = returns[sym]
        if ret.isna().sum() > len(ret) * 0.5:
            continue
        cagr = compute_cagr(px)
        ann_vol = ret.std() * np.sqrt(252)
        ann_ret = ret.mean() * 252
        sharpe = (ann_ret - RISK_FREE_RATE) / ann_vol if ann_vol > 0 else np.nan
        down = downside_std(ret) * np.sqrt(252)
        sortino = (ann_ret - RISK_FREE_RATE) / down if down > 0 else np.nan
        roll_max = px.cummax()
        dd = (px / roll_max) - 1
        max_dd = dd.min()
        calmar = ann_ret / abs(max_dd) if max_dd < 0 else np.nan
        common = ret.index.intersection(nifty_ret.index)
        aligned = pd.concat([ret.loc[common], nifty_ret.loc[common]], axis=1).dropna()
        beta = np.cov(aligned.iloc[:, 0], aligned.iloc[:, 1])[0, 1] / np.var(aligned.iloc[:, 1]) if len(aligned) > 100 else np.nan
        rec_days = rolling_recovery_days(px)
        results[sym] = {"CAGR_10Y": cagr, "Sharpe": sharpe, "Sortino": sortino, "Calmar": calmar,
                        "Volatility": ann_vol, "Max_Drawdown": max_dd, "Beta": beta, "Recovery_Days": rec_days}
    return pd.DataFrame(results).T

# SCORING SYSTEM
def normalize(v, cap): return max(0, min(v / cap, 1)) if pd.notna(v) else 0
def inverse(v, cap): return max(0, min(1 - v / cap, 1)) if pd.notna(v) else 0
def dd_norm(v): return max(0, min(1 - abs(v) / 0.6, 1)) if pd.notna(v) else 0
def beta_norm(v):
    if pd.isna(v): return 0.5
    v = float(v)
    if v <= 1.2: return 1 - abs(v - 1) / 0.4
    return max(0, 1 - (v - 1.2) / 0.8)

weights = {"CAGR_10Y": 25, "Sharpe": 15, "Sortino": 10, "Calmar": 10,
           "Volatility": 10, "Max_Drawdown": 10, "Beta": 5, "Recovery_Days": 5}

def score(row):
    s = 0
    s += normalize(row["CAGR_10Y"], 0.20) * weights["CAGR_10Y"]
    s += normalize(row["Sharpe"], 1.5) * weights["Sharpe"]
    s += normalize(row["Sortino"], 2.0) * weights["Sortino"]
    s += normalize(row["Calmar"], 1.5) * weights["Calmar"]
    s += inverse(row["Volatility"], 0.35) * weights["Volatility"]
    s += dd_norm(row["Max_Drawdown"]) * weights["Max_Drawdown"]
    s += beta_norm(row["Beta"]) * weights["Beta"]
    s += inverse(row["Recovery_Days"], 800) * weights["Recovery_Days"]
    return (s / sum(weights.values())) * 100

# EXECUTION (Notebook-Compatible)
try:
    sector = input("\n🔍 Enter the sector (e.g., IT, BANKING, PHARMA) or type 'ALL' to analyze all sectors: ").strip().upper()
except Exception:
    print("\n💡 Running in notebook mode — defaulting to 'IT' sector.")
    sector = "IT"

if sector == "ALL":
    for sec, tickers in SECTOR_COMPANIES.items():
        adj = download_adj_close(tickers)
        if adj.empty:
            print(f"\n⚠️ No data available for {sec}.")
            continue
        metrics = calculate_metrics(adj)
        metrics["Decision_Score"] = metrics.apply(score, axis=1)
        top5 = metrics.sort_values("Decision_Score", ascending=False).head(5)

        print(f"\n✅ {sec} Sector: 10-Year Analysis Complete!")
        for i, (sym, row) in enumerate(top5.iterrows(), start=1):
            print(f"🏅 {i}. {sym} — Score: {row['Decision_Score']:.2f}/100 | CAGR: {row['CAGR_10Y']*100:,.2f}% | Sharpe: {row['Sharpe']:.2f}")
        print("-" * 70)

else:
    if sector not in SECTOR_COMPANIES:
        print(f"\n⚠️ '{sector}' is not a valid sector. Please choose from: {', '.join(SECTOR_COMPANIES.keys())}")
    else:
        adj = download_adj_close(SECTOR_COMPANIES[sector])
        metrics = calculate_metrics(adj)
        metrics["Decision_Score"] = metrics.apply(score, axis=1)
        top5 = metrics.sort_values("Decision_Score", ascending=False).head(5)

        print(f"\n📊 Top 5 Companies in {sector} Sector (10-Year Performance):\n")
        for i, (sym, row) in enumerate(top5.iterrows(), start=1):
            print(f"🏅 {i}. {sym}")
            print(f"   📊 Overall Score: {row['Decision_Score']:.2f}/100")
            print(f"   💰 CAGR (10Y): {row['CAGR_10Y']*100:,.2f}%")
            print(f"   📈 Sharpe: {row['Sharpe']:.2f} | Sortino: {row['Sortino']:.2f} | Calmar: {row['Calmar']:.2f}")
            print(f"   🛡️ Volatility: {row['Volatility']*100:,.2f}% | Max DD: {row['Max_Drawdown']*100:,.2f}% | Beta: {row['Beta']:.2f}")
            print(f"   🕒 Recovery Days: {row['Recovery_Days']:.0f} days")
            print("-" * 70)

💼 Sectoral & Thematic Indices (NSE India)

🏦 Financial & Banking
NIFTY Bank
NIFTY Private Bank
NIFTY PSU Bank
NIFTY Financial Services
NIFTY Financial Services 25/50

💻 Technology & IT
NIFTY IT
NIFTY Digital Index (Thematic – focuses on digital transformation companies)

⚙️ Infrastructure & Industrial
NIFTY Infrastructure
NIFTY PSE (Public Sector Enterprises)
NIFTY CPSE (Central Public Sector Enterprises)
NIFTY Infrastructure Debt Fund

🛢️ Energy & Resources
NIFTY Energy
NIFTY Oil & Gas
NIFTY Commodities
NIFTY Metal

🏠 Consumer & Lifestyle
NIFTY FMCG
NIFTY Consumer Durables
NIFTY India Consumption

🏗️ Manufacturing & Engineering
NIFTY Auto
NIFTY Industrials
NIFTY Manufacturing

💊 Healthcare
NIFTY Healthcare Index
NIFTY Pharma

🌱 Thematic & Emerging Trends
NIFTY India Digital Index
NIFTY EV & New Age Automotive Index
NIFTY100 ESG Sector Leaders
NIFTY ESG 50
NIFTY Commodities
NIFTY MNC
NIFTY Dividend Opportunities 50
NIFTY High Beta 50
NIFTY Low Volatility 50

If you just want to invest 